# TensorFlow Eager Execution Tutorial

TensorFlow *Get Started with Eager Execution* tutorial: https://www.tensorflow.org/get_started/eager

## Setup the Environment

In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

In [ ]:
print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

## Import the Dataset

In [ ]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

In [ ]:
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

In [ ]:
print(f'Local copy of the dataset file: {train_dataset_fp}')

In [ ]:
# Take a peak at the contents of the csv file
i = 0
with open(train_dataset_fp) as f:
    for line in f:
        print(line)
        i += 1
        
        if i > 5:
            break